# How to Make Interactive Plots for EMIT

**This notebook is under development. A few caveats:
1. This uses a lot of memory 25+GB and may not run great locally, sometimes there is a delay with the visualization tools.
2. This has primarily tested using a JupyterHub cloud computing instance to access EMIT data via s3. 
3. This requires Earthdata Login credentials stored in a .netrc file.
4. Some portions of the plotting are hard-coded and currently only works with orthorectified data.

To use locally, you will need to download the granules you want (L2A Reflectance, L2A Reflectance Uncertainty, and L2A Mask) or stream using HTTPS (slow) and set them equal to the `fp` filepath variables the third cell below.

In [ ]:
!pip install spectral

In [ ]:
# Import Packages
import math
import requests
import pandas as pd
import datetime as dt
import geopandas as gp
from shapely.geometry import MultiPolygon, Polygon, box
import os
import sys
import s3fs
from osgeo import gdal
import numpy as np
import xarray as xr
import hvplot.xarray
import hvplot.pandas
import holoviews as hv
import geopandas as gpd
import panel as pn
sys.path.append('../modules/')
import emit_tools as et

Get AWS credentials

In [ ]:
# Get AWS credentials
temp_creds_req = requests.get('https://data.lpdaac.earthdatacloud.nasa.gov/s3credentials').json() # use lpdaac credential endpoint for EMIT data
# Create s3fs session
fs_s3 = s3fs.S3FileSystem(anon=False, 
                          key=temp_creds_req['accessKeyId'], 
                          secret=temp_creds_req['secretAccessKey'], 
                          token=temp_creds_req['sessionToken'])

Assign granules to objects - here to run locally, download these granules via earthdata search

In [ ]:
fp_rfl = fs_s3.open('s3://lp-prod-protected/EMITL2ARFL.001/EMIT_L2A_RFL_001_20230413T112349_2310308_001/EMIT_L2A_RFL_001_20230413T112349_2310308_001.nc', mode='rb')
fp_unc = fs_s3.open('s3://lp-prod-protected/EMITL2ARFL.001/EMIT_L2A_RFL_001_20230413T112349_2310308_001/EMIT_L2A_RFLUNCERT_001_20230413T112349_2310308_001.nc', mode='rb')
fp_mask = fs_s3.open('s3://lp-prod-protected/EMITL2ARFL.001/EMIT_L2A_RFL_001_20230413T112349_2310308_001/EMIT_L2A_MASK_001_20230413T112349_2310308_001.nc', mode='rb')

Open datasets in xarray using `emit_xarray` function. Take a look at `emit_tools.py` for details. This function rearranges and flattens the dataset,
and also offers an option to orthorectify using the provided GLT

In [ ]:
rfl  = et.emit_xarray(fp_rfl, ortho=True)
unc = et.emit_xarray(fp_unc, ortho=True)
# Merge Reflectance and Uncertainty Datasets
ds = xr.merge([rfl,unc])
del rfl
del unc
ds_mask = et.emit_xarray(fp_mask, ortho=True)

Quickly create a polygon to use for clipping using geopandas.

In [ ]:
lon_pts = [-3.91, -3.22, -3.085, -2.967, -3.061,-3.91]
lat_pts = [51.23, 51.16, 51.158, 51.247, 51.31, 51.23]
polygon_geom = Polygon(zip(lon_pts, lat_pts))
polygon = gpd.GeoDataFrame(index=[0], crs='epsg:4326', geometry=[polygon_geom])

Clip the data to our area of interest. This makes the data size more managable. The interactive plots will work without this step, but may be slow for several reasons, such as:
- large file size
- slow data streaming since EMIT is not chunked
- inefficiencies in code (probably mine)

In [ ]:
ds = ds.rio.clip(polygon.geometry.values,polygon.crs, all_touched=True)
ds_mask = ds_mask.rio.clip(polygon.geometry.values,polygon.crs, all_touched=True)

Calculate Min/Max reflectance using uncertainty for error bars in spectral plot.

In [ ]:
#Set DeepH2O absorption regions to NAN for better visual
ds['reflectance'].data[ds['reflectance'].data == -0.01] = np.nan

# Calculate Min and Max Reflectance values for visualization (error bars/area plot)
ds['min_refl'] = ds['reflectance']-ds['reflectance_uncertainty']
ds['max_refl'] = ds['reflectance']+ds['reflectance_uncertainty']
ds

The cell below builds a series of widgets that can be used to interact with the spatial plot (map) of the data. The map can then be clicked to display a spectral plot of the selected pixel, as well as an additional spectra where the cursor is hovering on the map.

To do:
- Remove hardcoding that limits to L2A Reflectance and Orthorectified only datasets.
- Better ways to control brightness/contrast?
- The RGB plotting method could likely be improved. The final goal is to be able to add a mask and also show a basemap. Currently, adding basemap tiles using hvplot/geoviews disrupts the functionality of the point and click spectral plot.
- Explore using hv.RGB instead of hvplot. The hvplot.rgb function doesn't seem to allow using a mask layer like the hv.RGB does, although using geoviews with the hv.RGB may not be possible. 

In [ ]:
# Function to build a new spectral plot based on mouse hover positional information retrieved from the RGB image using our full reflectance dataset
# Select Widgets
rgb_inds = np.array([np.nanargmin(abs(ds['wavelengths'].values - x)) for x in [650,560,470]])
wl_list = ds['wavelengths'].data.tolist()
wl_r = pn.widgets.Select(name='Wavelength 1 (R) (nm)',options=wl_list, value=wl_list[rgb_inds[0]])
wl_g = pn.widgets.Select(name='Wavelength 2 (G) (nm)',options=wl_list, value=wl_list[rgb_inds[1]])
wl_b = pn.widgets.Select(name='Wavelength 3 (B) (nm)',options=wl_list, value=wl_list[rgb_inds[2]])
min_pct = pn.widgets.IntSlider(name='Minimum Percentile', value=1, start=1, end=100)
max_pct = pn.widgets.IntSlider(name='Maximum Percentile', value=100, start=1, end=100)
gamma_int = pn.widgets.IntSlider(name='Image Gamma', value=50, start=1, end=100)
white_back = pn.widgets.Checkbox(name='White Backgroud')
unc_check = pn.widgets.Checkbox(name='Show Uncertainty')
# Define Function To Generate Map data using selection from widgets
@pn.depends(wl_r,wl_g,wl_b, min_pct, max_pct, gamma_int, white_back, unc_check)
def gen_map(wl_r, wl_g, wl_b, min_pct, max_pct, gamma_int, white_back, unc_check):
    
    # Create RGB Map
            
    # WL Selection
    sel_inds = np.array([np.nanargmin(abs(ds['wavelengths'].values - x)) for x in [wl_r,wl_g,wl_b]])       
    rgb = ds.sel(bands=(sel_inds))
    rgb_array = rgb['reflectance'].data
    #rgb_array = rgb_array.clip(0,1)        
    plot_title=f'{round(wl_r,2)}, {round(wl_g,2)}, {round(wl_b,2)} nm color image'
    
    # Gamma Correction to 3-band image

    rgb_array -= np.nanpercentile(rgb_array,min_pct,axis=(0,1))[np.newaxis,np.newaxis,:] # scale from 2-95 %
    rgb_array /= np.nanpercentile(rgb_array,max_pct,axis=(0,1))[np.newaxis,np.newaxis,:]
    rgb_array = rgb_array.clip(0,1)
    gamma = math.log((gamma_int/250))/math.log(np.nanmean(rgb_array)) # baseline = 0.2, will create a range of 0-0.4
    rgb_array = np.power(rgb_array,gamma).clip(0,1)
    if white_back:
        rgb_array= np.nan_to_num(rgb_array, nan=1) # Is there a better way to implement what is essentially a mask
    
    # Put back into xarray
    rgb['reflectance'].data = rgb_array
    
    # Drop excess variables and rename reflectance to RGB
    rgb = rgb.drop_vars(('fwhm','good_wavelengths','wavelengths'))
    rgb = rgb.rename({'reflectance':'rgb'})
    # Create Map to return   
    map = rgb.hvplot.rgb(x='longitude',y='latitude', bands='bands', geo=True, frame_width=400, frame_height=400).opts(title=plot_title)  # use aspect='equal' option?
    
    # Add interactive spectral plot 
    # Function to build a new spectral plot based on mouse hover positional information retrieved from the RGB image using our full reflectance dataset 
    def point_spectra(x,y):
        point = ds.sel(longitude=x,latitude=y,method='nearest')
        pointer_rfl_plot = point.hvplot.line(y='reflectance',x='wavelengths', frame_width=400)
        if unc_check:
            pointer_unc_plot = point.hvplot.area(x='wavelengths', y='min_refl', y2='max_refl', alpha=0.2)
            point_out = pointer_rfl_plot.opts(title = f'Latitude = {point.latitude.values.round(3)}, Longitude = {point.longitude.values.round(3)}')*pointer_unc_plot
        else:
            point_out = pointer_rfl_plot.opts(title = f'Latitude = {point.latitude.values.round(3)}, Longitude = {point.longitude.values.round(3)}')            
        return point_out

    # Function to build spectral plot of clicked location to show on hover stream plot
    def click_spectra(x,y):
        clicked = ds.sel(longitude=x,latitude=y,method='nearest')
        clicked_rfl_plot = clicked.hvplot.line(y='reflectance',x='wavelengths', frame_width=400)
        if unc_check:
            clicked_unc_plot = clicked.hvplot.area(x='wavelengths', y='min_refl', y2='max_refl', alpha=0.2)
            clicked_out = clicked_rfl_plot*clicked_unc_plot.opts(title = f'Latitude = {clicked.latitude.values.round(3)}, Longitude = {clicked.longitude.values.round(3)}')
        else:
            clicked_out = clicked_rfl_plot.opts(title = f'Latitude = {clicked.latitude.values.round(3)}, Longitude = {clicked.longitude.values.round(3)}')
        return clicked_out 
    
    # Stream of X and Y positional data
    posxy = hv.streams.PointerXY(source=map, x=-2.1, y=16.86) 
    clickxy = hv.streams.Tap(source=map, x=-2.1, y=16.86)   
    
    # Define the Dynamic Maps
    point_dmap = hv.DynamicMap(point_spectra, streams=[posxy])
    click_dmap = hv.DynamicMap(click_spectra, streams=[clickxy])
    
    return map+point_dmap*click_dmap
pn.Row(pn.WidgetBox(wl_r,wl_g,wl_b, min_pct, max_pct, gamma_int, white_back, unc_check), gen_map)   

Single Layer Interactive Plot with Masking Note: basemap breaks interactive plotting - not sure whats going on with addition of tiles that causes this.

In [ ]:
# Function to build a new spectral plot based on mouse hover positional information retrieved from the RGB image using our full reflectance dataset 
def point_spectra(x,y):
    point = ds.sel(longitude=x,latitude=y,method='nearest')
    pointer_rfl_plot = point.hvplot.line(y='reflectance',x='wavelengths', frame_width=400)
    pointer_unc_plot = point.hvplot.area(x='wavelengths', y='min_refl', y2='max_refl', alpha=0.2)
    point_out = pointer_rfl_plot.opts(title = f'Latitude = {point.latitude.values.round(3)}, Longitude = {point.longitude.values.round(3)}')*pointer_unc_plot
    return point_out

# Function to build spectral plot of clicked location to show on hover stream plot
def click_spectra(x,y):
    clicked = ds.sel(longitude=x,latitude=y,method='nearest')
    clicked_rfl_plot = clicked.hvplot.line(y='reflectance',x='wavelengths', frame_width=400)
    clicked_unc_plot = clicked.hvplot.area(x='wavelengths', y='min_refl', y2='max_refl', alpha=0.2)
    clicked_out = clicked_rfl_plot*clicked_unc_plot.opts(title = f'Latitude = {clicked.latitude.values.round(3)}, Longitude = {clicked.longitude.values.round(3)}')
    return clicked_out

# Preliminary Mask Selection List
mask_layer_list = ['None']+ds_mask['mask_bands'].data.tolist()
# AOD and H2O are not masks, they have values - We could add this as mask layers, but we should allow users 
## to provide the threshold values because they will vary based on the type of research
mask_rm = ['AOD550','H2O (g cm-2)']
mask_layer_list = [m for m in mask_layer_list if m not in mask_rm]

# Select Widgets
wavelength = pn.widgets.Select(name='Wavelength (nm)',options=ds['wavelengths'].data.tolist())
mask_name = pn.widgets.Select(name='Mask Band',options=mask_layer_list)
basemap = pn.widgets.Checkbox(name='Show Basemap')
alpha = pn.widgets.IntSlider(name='Transparency', value=100, start=0, end=100)

# Define Function To Generate Map data using selection from widgets
@pn.depends(wavelength, basemap, alpha, mask_name)
def gen_map(wavelength, basemap, alpha, mask_name):
    
    # WL Selection
    band_index = np.nanargmin(abs(ds['wavelengths'].values-wavelength))
    layer_ds = ds['reflectance'].sel(bands=band_index)
    plot_title=f'{wavelength} nm'
    
    # Mask Selection
    
    # How to handle added index of none
    if mask_name == 'None':
        mask_layer = ds_mask['mask'].sel(bands=0)
        mask_layer.data = np.full((mask_layer.data.shape[0],mask_layer.data.shape[1]),1,dtype='int')
    
    # How to handle non-AOD and non-H2O  - Note masks need to be inverted to be applied using the method below
    else: 
        mask_index = ds_mask['mask_bands'].data.tolist().index(mask_name)
        mask_layer = ds_mask['mask'].sel(bands=mask_index)
        mask_layer.data = np.nan_to_num(mask_layer.data).astype('int')
        # Invert and apply mask
        mask_layer.data = np.where((mask_layer.data==0)|(mask_layer.data==1), mask_layer.data^1, mask_layer.data)
        
    
    # BaseMap - additional tiles could be used or added as a selection instead of checkbox
    if basemap:
        tile = 'EsriImagery'
    else:
        tile = None
    
    # Create Map to return
    map = layer_ds.where(mask_layer).hvplot.image(x='longitude',y='latitude',cmap='viridis', geo=True, tiles=tile, frame_width=400, alpha=(alpha/100)).opts(title=plot_title)
    
    # Stream of X and Y positional data
    posxy = hv.streams.PointerXY(source=map, x=-2.1, y=16.86) 
    clickxy = hv.streams.Tap(source=map, x=-2.1, y=16.86)
    
    # Stream of X and Y positional data
    posxy = hv.streams.PointerXY(source=map, x=-2.1, y=16.86) 
    clickxy = hv.streams.Tap(source=map, x=-2.1, y=16.86)
    
    # Define the Dynamic Maps
    point_dmap = hv.DynamicMap(point_spectra, streams=[posxy])
    click_dmap = hv.DynamicMap(click_spectra, streams=[clickxy])
    
    return map+point_dmap*click_dmap
pn.Row(pn.WidgetBox('## Layer Comparison',wavelength, mask_name, alpha, basemap), gen_map)